In [20]:
import numpy as np
import pandas as pd

import seaborn as sns

## Load the data from the Excel sheets

In [21]:
ANIME_CSV_PATH = "./datasets/anime.csv"
RATING_CSV_PATH = "./datasets/rating.csv"

In [22]:
anime_df = pd.read_csv(ANIME_CSV_PATH)
rating_df = pd.read_csv(RATING_CSV_PATH)

## Preprocess the data

In [23]:
rating_df.rating.replace({-1: np.nan}, inplace=True)

In [24]:
anime_tv_df = anime_df[anime_df['type'] == 'TV']

In [25]:
merged_df = pd.merge(anime_tv_df, rating_df, on='anime_id', suffixes=['_anime', '_user'])

In [26]:
merged_df = merged_df[['user_id', 'name', 'rating_user']]

In [27]:
# sns.boxplot(y=merged_df.groupby('name').count()[merged_df.groupby('name').count()['rating_user'] > 5000]['rating_user'])

In [28]:
merged_limit_df = merged_df[merged_df.user_id <= 1000]

create pivot table for merged data

In [29]:
user_anime_matrix = merged_limit_df.pivot_table(index=['user_id'], columns=['name'], values='rating_user')

In [30]:
norm_user_anime_matrix = user_anime_matrix.apply(lambda x: (x-np.mean(x))/(np.max(x)-np.min(x)), axis=1)

In [31]:
norm_user_anime_matrix.fillna(0, inplace=True)
norm_user_anime_matrix = norm_user_anime_matrix.T
norm_user_anime_matrix = norm_user_anime_matrix.loc[:, (norm_user_anime_matrix != 0).any(axis=0)]

## Pearson Correlation

In [32]:
item_similarity = norm_user_anime_matrix.T.corr(method='pearson')
user_similarity = norm_user_anime_matrix.corr(method='pearson')

In [33]:
anime_name = 'Naruto'
item_similarity[anime_name].sort_values(ascending=False).iloc[:20]

name
Naruto                                  1.000000
Hoshi no Kirby                          0.181727
Sansha Sanyou                           0.178296
Venus Project: Climax                   0.171696
Stitch!                                 0.171696
Domo TV                                 0.171696
Suzakinishi the Animation               0.171696
Pan de Peace!                           0.166731
Concrete Revolutio: Choujin Gensou      0.163048
Kuromukuro                              0.163021
Classroom☆Crisis                        0.150690
Soul Link                               0.146919
Bishoujo Yuugi Unit Crane Game Girls    0.143040
Lime-iro Senkitan                       0.141570
Spider Riders: Yomigaeru Taiyou         0.136976
Saiko Robot Kombock                     0.127470
Fractale                                0.118590
Avenger                                 0.117764
Popotan                                 0.115998
Pokemon Advanced Generation             0.114676
Name: Naruto, d

In [34]:
user_similarity[5].sort_values(ascending=False).iloc[:20]

user_id
5      1.000000
529    0.223348
771    0.222064
38     0.213314
687    0.198078
392    0.187884
29     0.187511
148    0.186629
250    0.181672
776    0.178238
702    0.178048
875    0.172617
500    0.172313
920    0.170556
567    0.162137
235    0.160802
926    0.157492
183    0.156441
189    0.155406
139    0.150197
Name: 5, dtype: float64